In [48]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import os
from scipy import stats
import plotly.express as px


In [4]:
df = pd.read_csv("RawData_house_sale.csv", index_col = 0)
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19980 entries, 0 to 19979
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   url                      19736 non-null  object 
 1   id                       19736 non-null  float64
 2   region                   19697 non-null  object 
 3   province                 19697 non-null  object 
 4   locality                 19736 non-null  object 
 5   zip_code                 19736 non-null  float64
 6   Longitude                16784 non-null  float64
 7   Latitude                 16784 non-null  float64
 8   property_type            19736 non-null  object 
 9   property_subtype         19736 non-null  object 
 10  price                    18794 non-null  float64
 11  number_rooms             18795 non-null  float64
 12  living_area              17900 non-null  float64
 13  kitchen                  17524 non-null  object 
 14  furnished                67

In [53]:
# Calculate the total number of missing values per column
missing_values = df.isnull().sum().sort_values()

# Calculate the total number of entries (rows) in the DataFrame
total_entries = len(df)

# Calculate the proportion of missing values per column
proportion_missing = (missing_values / total_entries) * 100

# Print the proportion of missing values per column
print("Proportion of missing values per column:")
print(proportion_missing)

# Create a histogram using Plotly Express
fig = px.bar(missing_values, x=missing_values.index, y=missing_values.values)
fig.update_layout(
    title='Histogram of Null Values per Column',
    xaxis_title='Column Name',
    yaxis_title='Number of Null Values'
)
fig.show()

Proportion of missing values per column:
url                         0.000000
id                          0.000000
locality                    0.000000
zip_code                    0.000000
property_subtype            0.000000
property_type               0.000000
region                      0.209655
province                    0.209655
number_rooms                4.578422
price                       4.584089
living_area                 9.168178
kitchen                    11.134406
Longitude                  15.157525
Latitude                   15.157525
EPC_score                  15.746827
EnergyConsumptionPerSqm    17.645059
surface_land               23.770399
number_facades             23.974388
building_state             26.609248
energy_type                28.286491
terrace                    44.622620
parking_indoor             55.779692
parking_outdoor            64.959202
furnished                  65.508840
terrace_area               66.245467
swimming_pool              66.4777

**Remove duplicates and empty rows**

In [5]:
# Remove duplicates
df.drop_duplicates(inplace=True)
# reset the index
df.reset_index(drop = True, inplace = True)
df.head()
df.shape

(17649, 29)

In [6]:
# Remove all the empty rows
rows_to_remove = []
# Iterate through each row in the DataFrame
for index, row in df.iterrows():
# Check if all values in the row are null
    if row.isnull().all():
        # If all values are null, mark the index for removal
        rows_to_remove.append(index)
# Remove the marked rows from the DataFrame
df = df.drop(rows_to_remove)
df.to_csv('cleaned_house.csv')
# df.to_csv(r'data\cleaned_house_house.csv')

# df_clean = pd.read_csv(r'data\cleaned_house_house.csv', index_col=0, low_memory=False)
df_clean = pd.read_csv('cleaned_house.csv', index_col=0)
df_clean


,url,id,region,province,locality,zip_code,Longitude,Latitude,property_type,property_subtype,...,garden_area,surface_land,number_facades,swimming_pool,building_state,energy_type,EPC_score,EnergyConsumptionPerSqm,parking_outdoor,parking_indoor
0,https://www.immoweb.be/en/classified/house/for...,11128213.0,Flanders,East Flanders,Wetteren,9230.0,3.884853,51.011410,HOUSE,HOUSE,...,NaN,90.0,2.0,False,GOOD,GAS,B,193.0,NaN,NaN
1,https://www.immoweb.be/en/classified/house/for...,11130920.0,Flanders,East Flanders,Ronse,9600.0,3.609445,50.740380,HOUSE,HOUSE,...,350.0,555.0,3.0,False,TO_BE_DONE_UP,FUELOIL,F,737.0,3.0,1.0
2,https://www.immoweb.be/en/classified/house/for...,11119431.0,Flanders,East Flanders,Ronse,9600.0,3.613201,50.737229,HOUSE,HOUSE,...,101.0,196.0,2.0,False,AS_NEW,GAS,A,57.0,NaN,NaN
3,https://www.immoweb.be/en/classified/house/for...,11132583.0,Wallonie,Liège,Neupré,4121.0,5.488573,50.548515,HOUSE,HOUSE,...,500.0,601.0,3.0,False,GOOD,FUELOIL,E,348.0,NaN,1.0
4,https://www.immoweb.be/en/classified/house/for...,10874763.0,Wallonie,Liège,Ans,4430.0,5.519029,50.660049,HOUSE,HOUSE,...,110.0,261.0,2.0,False,GOOD,GAS,G,577.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17644,https://www.immoweb.be/en/classified/house/for...,11151064.0,Flanders,East Flanders,SINT-NIKLAAS,9100.0,4.163084,51.164513,HOUSE,HOUSE,...,37.0,178.0,2.0,NaN,GOOD,GAS,D,366.0,NaN,18.0
17645,https://www.immoweb.be/en/classified/house/for...,11151063.0,Wallonie,Hainaut,Dour,7370.0,NaN,NaN,HOUSE,HOUSE,...,NaN,127.0,3.0,NaN,TO_RESTORE,GAS,F,427.0,NaN,NaN
17646,https://www.immoweb.be/en/classified/apartment...,11151062.0,Brussels,Brussels,Ganshoren,1083.0,NaN,NaN,APARTMENT,APARTMENT,...,NaN,NaN,2.0,NaN,NaN,GAS,C,141.0,NaN,NaN
17647,https://www.immoweb.be/en/classified/apartment...,11151061.0,Brussels,Brussels,BERCHEM-SAINTE-AGATHE,1082.0,NaN,NaN,APARTMENT,APARTMENT,...,NaN,NaN,2.0,NaN,TO_RENOVATE,GAS,NaN,NaN,NaN,1.0


**Removing all rows without price as an dependent variable**

In [7]:
df_clean['price'].isna().value_counts()

# Remove rows where the "price" column is null
df_clean.dropna(subset=['price'], inplace=True)
df_clean

,url,id,region,province,locality,zip_code,Longitude,Latitude,property_type,property_subtype,...,garden_area,surface_land,number_facades,swimming_pool,building_state,energy_type,EPC_score,EnergyConsumptionPerSqm,parking_outdoor,parking_indoor
0,https://www.immoweb.be/en/classified/house/for...,11128213.0,Flanders,East Flanders,Wetteren,9230.0,3.884853,51.011410,HOUSE,HOUSE,...,NaN,90.0,2.0,False,GOOD,GAS,B,193.0,NaN,NaN
1,https://www.immoweb.be/en/classified/house/for...,11130920.0,Flanders,East Flanders,Ronse,9600.0,3.609445,50.740380,HOUSE,HOUSE,...,350.0,555.0,3.0,False,TO_BE_DONE_UP,FUELOIL,F,737.0,3.0,1.0
2,https://www.immoweb.be/en/classified/house/for...,11119431.0,Flanders,East Flanders,Ronse,9600.0,3.613201,50.737229,HOUSE,HOUSE,...,101.0,196.0,2.0,False,AS_NEW,GAS,A,57.0,NaN,NaN
3,https://www.immoweb.be/en/classified/house/for...,11132583.0,Wallonie,Liège,Neupré,4121.0,5.488573,50.548515,HOUSE,HOUSE,...,500.0,601.0,3.0,False,GOOD,FUELOIL,E,348.0,NaN,1.0
4,https://www.immoweb.be/en/classified/house/for...,10874763.0,Wallonie,Liège,Ans,4430.0,5.519029,50.660049,HOUSE,HOUSE,...,110.0,261.0,2.0,False,GOOD,GAS,G,577.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17644,https://www.immoweb.be/en/classified/house/for...,11151064.0,Flanders,East Flanders,SINT-NIKLAAS,9100.0,4.163084,51.164513,HOUSE,HOUSE,...,37.0,178.0,2.0,NaN,GOOD,GAS,D,366.0,NaN,18.0
17645,https://www.immoweb.be/en/classified/house/for...,11151063.0,Wallonie,Hainaut,Dour,7370.0,NaN,NaN,HOUSE,HOUSE,...,NaN,127.0,3.0,NaN,TO_RESTORE,GAS,F,427.0,NaN,NaN
17646,https://www.immoweb.be/en/classified/apartment...,11151062.0,Brussels,Brussels,Ganshoren,1083.0,NaN,NaN,APARTMENT,APARTMENT,...,NaN,NaN,2.0,NaN,NaN,GAS,C,141.0,NaN,NaN
17647,https://www.immoweb.be/en/classified/apartment...,11151061.0,Brussels,Brussels,BERCHEM-SAINTE-AGATHE,1082.0,NaN,NaN,APARTMENT,APARTMENT,...,NaN,NaN,2.0,NaN,TO_RENOVATE,GAS,NaN,NaN,NaN,1.0


**Exploring and Cleaning columns**

In [8]:
column_name = df_clean.columns
print(column_name)

def analyze_column(df_clean, column_name):
    column = df_clean[column_name]
    num_nan = column.isnull().sum()
    num_unique = column.nunique()
    unique_values = column.unique()
    percentage_nan = (num_nan / len(column)) * 100
    percentage_unique = (num_unique / len(column)) * 100
    print(f"Analysis for column '{column_name}':")
    print(f"Number of NaN values: {num_nan}")
    print(f"Number of unique values: {num_unique}")
    print(f"Percentage of NaN values: {percentage_nan:.2f}%")
    print(f"Percentage of unique values: {percentage_unique:.2f}%")
    print(f"Unique values: {unique_values}")


Index(['url', 'id', 'region', 'province', 'locality', 'zip_code', 'Longitude',
       'Latitude', 'property_type', 'property_subtype', 'price',
       'number_rooms', 'living_area', 'kitchen', 'furnished', 'fireplace',
       'terrace', 'terrace_area', 'garden', 'garden_area', 'surface_land',
       'number_facades', 'swimming_pool', 'building_state', 'energy_type',
       'EPC_score', 'EnergyConsumptionPerSqm', 'parking_outdoor',
       'parking_indoor'],
      dtype='object')


**Filling missing value with None for some columns**

In [9]:
df_clean.furnished = df_clean.furnished.fillna("None")
df_clean.fireplace = df_clean.fireplace.fillna("None")
df_clean.swimming_pool = df_clean.swimming_pool.fillna("None")
df_clean.parking_indoor = df_clean.parking_indoor.fillna("None")
df_clean.parking_outdoor = df_clean.parking_outdoor.fillna("None")
df_clean.terrace_area = df_clean.terrace_area.fillna("None")
df_clean.building_state = df_clean.building_state.fillna("None")
df_clean.Latitude = df.Latitude.fillna("None")
df_clean.Longitude = df.Longitude.fillna("None")
df_clean.kitchen = df.kitchen.fillna("None")
df_clean.surface_land = df.surface_land.fillna("None")

**Garden and Garden area** 

In [10]:
# Fill missing values in "Garden" column with False
df_clean['garden'] = df_clean['garden'].fillna(False)

# Change the data type of "Garden" to bool
df_clean['garden'] = df_clean['garden'].astype(bool)

# Fill missing values in 'Garden_area' column with 0
df_clean['garden_area'] = df_clean['garden_area'].fillna(0)

# Replace 'None' with 0 in the 'garden_area' and 'terrace_area' columns
df_clean['garden_area'].replace("None", 0, inplace=True)
df_clean['terrace_area'].replace("None", 0, inplace=True)


C:\Users\becod\AppData\Local\Temp\ipykernel_8076\2966937682.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_clean['garden'] = df_clean['garden'].fillna(False)
C:\Users\becod\AppData\Local\Temp\ipykernel_8076\2966937682.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean['garden_area'].replace("

**Terrace**

In [11]:
# fill missing values in "Terrace" column with False
df_clean['terrace'] = df_clean['terrace'].fillna(False)

C:\Users\becod\AppData\Local\Temp\ipykernel_8076\1162921177.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_clean['terrace'] = df_clean['terrace'].fillna(False)


**Furnished**

In [12]:
furnished = analyze_column(df_clean, "furnished")

Analysis for column 'furnished':
Number of NaN values: 0
Number of unique values: 3
Percentage of NaN values: 0.00%
Percentage of unique values: 0.02%
Unique values: [False 'None' True]


In [13]:
df_clean['furnished'] = df_clean['furnished'].replace(np.nan, False)
df_clean['furnished'] = df_clean['furnished'].replace(False, 0)
df_clean['furnished'] = df_clean['furnished'].replace(True, 1)
analyze_column(df_clean, 'furnished')

Analysis for column 'furnished':
Number of NaN values: 0
Number of unique values: 3
Percentage of NaN values: 0.00%
Percentage of unique values: 0.02%
Unique values: [0 'None' 1]


In [14]:
# Turn True/False into 1/0
for i in ["furnished", "terrace", "garden"]:
    print(df_clean[[i]].value_counts())
    df_clean[i] = df_clean[i].map({'True': 1, 'False': 0, '0':0, 0:0, 1:1})
    print(df_clean[[i]].value_counts())
#display(df_clean)

furnished
None         10753
0             5774
1              312
Name: count, dtype: int64
furnished
0.0          5774
1.0           312
Name: count, dtype: int64
terrace
True       9773
False      7066
Name: count, dtype: int64
terrace
1          9773
0          7066
Name: count, dtype: int64
garden
False     11080
True       5759
Name: count, dtype: int64
garden
0         11080
1          5759
Name: count, dtype: int64


**Number of facades**

In [15]:
df_clean["number_facades"].mode()
df_clean["number_facades"].value_counts()

# Fill missing values in 'number_facades' column with 2
df_clean['number_facades'].fillna(2, inplace=True)

df_clean.number_facades.value_counts()


C:\Users\becod\AppData\Local\Temp\ipykernel_8076\1634433712.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean['number_facades'].fillna(2, inplace=True)


number_facades
2.0    9006
4.0    4368
3.0    3439
1.0      17
6.0       3
5.0       3
8.0       2
7.0       1
Name: count, dtype: int64

**Surface of the land**

In [16]:
# Surface of the land has 8673 Nan entries
df_clean.surface_land.value_counts()

# Replace 'NaN' and 'None' with 0 in the 'surface_land' column
df_clean['surface_land'].replace({'NaN': 0, 'None': 0}, inplace=True)

df_clean.surface_land.value_counts()


C:\Users\becod\AppData\Local\Temp\ipykernel_8076\3186470020.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean['surface_land'].replace({'NaN': 0, 'None': 0}, inplace=True)
C:\Users\becod\AppData\Local\Temp\ipykernel_8076\3186470020.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_cle

surface_land
0.0        4377
150.0        78
140.0        76
120.0        75
100.0        74
           ... 
14739.0       1
4670.0        1
5635.0        1
2544.0        1
3125.0        1
Name: count, Length: 2514, dtype: int64

**State of the building**

In [17]:
# State of the building data has 'Nan'
df_clean.building_state.value_counts()

building_state
GOOD              5752
None              3887
AS_NEW            2666
TO_RENOVATE       2094
TO_BE_DONE_UP     1600
JUST_RENOVATED     736
TO_RESTORE         104
Name: count, dtype: int64

In [18]:
# Replace 'None' and "Nan" by 'unknown'
df_clean['building_state'] = df_clean['building_state'].replace({"None": "unknown"})
df_clean['building_state'] = df_clean['building_state'].replace({"Nan": "unknown"})
df_clean.building_state.value_counts()

building_state
GOOD              5752
unknown           3887
AS_NEW            2666
TO_RENOVATE       2094
TO_BE_DONE_UP     1600
JUST_RENOVATED     736
TO_RESTORE         104
Name: count, dtype: int64

**Kitchen**

In [19]:
# Kitchen data has about 3978 '0' and 1520 "None"
df_clean.kitchen.value_counts()
df_clean['kitchen'] = df_clean['kitchen'].replace({"0": "unknown", "None": "unknown"})

**living area**

In [20]:
living_area = analyze_column(df_clean, "living_area")
df_clean["living_area"].value_counts()

Analysis for column 'living_area':
Number of NaN values: 810
Number of unique values: 713
Percentage of NaN values: 4.81%
Percentage of unique values: 4.23%
Unique values: [  90.  156.  165.  167.  213.  430.  170.  169.  110.  114.  155.  162.
  163.  175.  135.  204.  143.  339.  171.  267.  179.  262.  130.  203.
  177.  123.  132.  140.  215.  360.   94.  134.  260.  384.  235.  120.
  160.  166.   86.  200.  230.  136.  100.  112.  147.  128.  142.  150.
  261.  188.  220.  207.  208.  202.  191.  394.  148.  137.  108.  180.
  313.  190.   96. 1200.  248.   83.   72.   nan  131.  151.   80.   75.
  121.   91.  269.  325. 1350.  158.  370.  106.  336.  300.  285.  198.
  382.  480.  126.  187.  196.  209.  185.  194.  205.  228.  105.  420.
  153.  231.  268.   38.  250.  176.   98.   40.  111.  168.  255.   84.
  445.   45.   41.  210.  125.  102.  138.   81.  172.  161.  159.  154.
   78.  122.   97.  192.  273.  547.  270.  149.  217.  334.  630.  223.
  304.  118.  232.  227. 

living_area
150.0     277
120.0     242
200.0     236
160.0     217
180.0     206
         ... 
1608.0      1
868.0       1
574.0       1
371.0       1
725.0       1
Name: count, Length: 713, dtype: int64

In [21]:
# Drop rows where the 'living_area' column contains NaN values
df_clean.dropna(subset=['living_area'], inplace=True)
df_clean.living_area.value_counts()

living_area
150.0     277
120.0     242
200.0     236
160.0     217
180.0     206
         ... 
1608.0      1
868.0       1
574.0       1
371.0       1
725.0       1
Name: count, Length: 713, dtype: int64

**EPC_score**

In [22]:
EPC_score = analyze_column(df_clean, "EPC_score")
df_clean["EPC_score"].value_counts()

Analysis for column 'EPC_score':
Number of NaN values: 1932
Number of unique values: 11
Percentage of NaN values: 12.05%
Percentage of unique values: 0.07%
Unique values: ['B' 'F' 'A' 'E' 'G' 'C' 'D' nan 'A+' 'A++' 'F_E' 'A+_A++']


EPC_score
C         2767
D         2551
B         2497
F         2269
E         1849
G         1043
A          996
A+          96
A++         26
A+_A++       2
F_E          1
Name: count, dtype: int64

In [23]:
df_clean["EPC_score"]=np.where(df_clean["EPC_score"].isin(['F_E', 'A_A+']), "Not specified", df_clean["EPC_score"])

In [24]:
df_clean.dropna(subset=['EPC_score'], inplace=True)
df_clean.EPC_score.value_counts()

EPC_score
C                2767
D                2551
B                2497
F                2269
E                1849
G                1043
A                 996
A+                 96
A++                26
A+_A++              2
Not specified       1
Name: count, dtype: int64

**Energy Consumption per Sqm**

In [25]:
EnergyConsumptionPerSqm = analyze_column(df_clean, "EnergyConsumptionPerSqm")

Analysis for column 'EnergyConsumptionPerSqm':
Number of NaN values: 387
Number of unique values: 1115
Percentage of NaN values: 2.75%
Percentage of unique values: 7.91%
Unique values: [ 193.    737.     57.   ... 1006.   1045.     43.54]


In [26]:
df_clean.dropna(subset=['EnergyConsumptionPerSqm'], inplace=True)
df_clean.EnergyConsumptionPerSqm.value_counts()

EnergyConsumptionPerSqm
251.0     53
198.0     52
255.0     49
273.0     48
164.0     48
          ..
839.0      1
978.0      1
1304.0     1
1085.0     1
1356.0     1
Name: count, Length: 1115, dtype: int64

**Energy type**

In [27]:
energy_type = analyze_column(df_clean,"energy_type")
df_clean["energy_type"].value_counts()

Analysis for column 'energy_type':
Number of NaN values: 2543
Number of unique values: 7
Percentage of NaN values: 18.55%
Percentage of unique values: 0.05%
Unique values: ['GAS' 'FUELOIL' 'PELLET' nan 'ELECTRIC' 'WOOD' 'CARBON' 'SOLAR']


energy_type
GAS         7436
FUELOIL     2829
ELECTRIC     660
PELLET       162
WOOD          57
CARBON        18
SOLAR          5
Name: count, dtype: int64

In [28]:
df_clean['energy_type'] = df_clean['energy_type'].fillna("unknown")
df_clean['energy_type'] = df_clean['energy_type'].replace({"Nan": "unknown"})
df_clean["energy_type"].value_counts()

energy_type
GAS         7436
FUELOIL     2829
unknown     2543
ELECTRIC     660
PELLET       162
WOOD          57
CARBON        18
SOLAR          5
Name: count, dtype: int64

**Locality, province and zip codes**

In [29]:
# leaving province as it is
province = analyze_column(df_clean,"province")
df_clean["province"].value_counts()


Analysis for column 'province':
Number of NaN values: 2
Number of unique values: 11
Percentage of NaN values: 0.01%
Percentage of unique values: 0.08%
Unique values: ['East Flanders' 'Liège' 'Luxembourg' 'Hainaut' 'Walloon Brabant'
 'Antwerp' 'West Flanders' 'Brussels' 'Flemish Brabant' 'Limburg' 'Namur'
 nan]


province
Antwerp            2286
East Flanders      1999
West Flanders      1818
Liège              1544
Brussels           1298
Flemish Brabant    1279
Hainaut            1229
Walloon Brabant     797
Luxembourg          563
Namur               498
Limburg             397
Name: count, dtype: int64

In [30]:
# leaving locality as it is 
locality = analyze_column(df_clean, "locality")
df_clean["locality"].value_counts()

Analysis for column 'locality':
Number of NaN values: 0
Number of unique values: 2443
Percentage of NaN values: 0.00%
Percentage of unique values: 17.82%
Unique values: ['Wetteren' 'Ronse' 'Neupré' ... 'Eggewaartskapelle' 'Hoogstraten Meer'
 'Pussemange']


locality
Antwerp              262
Gent                 227
Uccle                176
Liège                163
Seraing              148
                    ... 
AUBY-SUR-SEMOIS        1
VISE                   1
WIDEUMONT VILLAGE      1
MEULEBEKE              1
Pussemange             1
Name: count, Length: 2443, dtype: int64

In [31]:
zip_code = analyze_column(df_clean, "zip_code")
df_clean.zip_code.max()

Analysis for column 'zip_code':
Number of NaN values: 0
Number of unique values: 947
Percentage of NaN values: 0.00%
Percentage of unique values: 6.91%
Unique values: [9230. 9600. 4121. 4430. 4032. 4219. 4530. 4100. 6997. 7910. 1470. 1495.
 4671. 9000. 2100. 6940. 8510. 4257. 4540. 4260. 6780. 1000. 1800. 2600.
 9500. 1640. 4300. 9340. 4030. 1410. 1070. 1745. 6250. 1300. 1330. 8501.
 1933. 4500. 2140. 2310. 4040. 8790. 9700. 9260. 2223. 4287. 7063. 7170.
 7610. 4520. 9200. 1420. 6637. 6838. 2880. 2800. 9690. 1830. 2060. 2070.
 8940. 2018. 4470. 9100. 9050. 4800. 7521. 4122. 1190. 1731. 9031. 7500.
 8710. 9630. 9840. 8850. 4400. 1435. 2610. 3740. 9032. 1030. 4650. 9620.
 1120. 2000. 4101. 4420. 4683. 9300. 9660. 6792. 2970. 8670. 2020. 2570.
 2620. 4684. 4870. 6900. 2845. 9120. 4102. 1490. 4051. 6890. 4682. 4350.
 2910. 4690. 4900. 2240. 5000. 5100. 2870. 4841. 1180. 2980. 8490. 9310.
 2812. 9831. 6061. 7134. 2820. 1090. 4141. 5101. 1301. 5190. 8900. 1440.
 1500. 6560. 4432. 8630. 3570.

9991.0

**Property type and subtype**

In [32]:
property_type = analyze_column(df_clean, "property_type")
# type of property(house/apartment)
# apartment group = 1545
# house group = 232

df_clean['property_type'].value_counts()



Analysis for column 'property_type':
Number of NaN values: 0
Number of unique values: 2
Percentage of NaN values: 0.00%
Percentage of unique values: 0.01%
Unique values: ['HOUSE' 'APARTMENT']


property_type
HOUSE        11326
APARTMENT     2384
Name: count, dtype: int64

In [33]:
property_subtype = analyze_column(df_clean, "property_subtype")
df_clean["property_subtype"].value_counts()

Analysis for column 'property_subtype':
Number of NaN values: 0
Number of unique values: 23
Percentage of NaN values: 0.00%
Percentage of unique values: 0.17%
Unique values: ['HOUSE' 'VILLA' 'MIXED_USE_BUILDING' 'APARTMENT_BLOCK'
 'EXCEPTIONAL_PROPERTY' 'BUNGALOW' 'MANSION' 'APARTMENT' 'TOWN_HOUSE'
 'COUNTRY_COTTAGE' 'PENTHOUSE' 'LOFT' 'FLAT_STUDIO' 'OTHER_PROPERTY'
 'GROUND_FLOOR' 'MANOR_HOUSE' 'DUPLEX' 'CHALET' 'SERVICE_FLAT' 'FARMHOUSE'
 'CASTLE' 'TRIPLEX' 'KOT']


property_subtype
HOUSE                   8180
APARTMENT               1866
VILLA                   1239
APARTMENT_BLOCK          519
MIXED_USE_BUILDING       492
EXCEPTIONAL_PROPERTY     198
MANSION                  185
TOWN_HOUSE               150
DUPLEX                   147
BUNGALOW                 137
GROUND_FLOOR             126
FLAT_STUDIO              109
COUNTRY_COTTAGE           80
PENTHOUSE                 78
FARMHOUSE                 55
MANOR_HOUSE               48
CHALET                    25
TRIPLEX                   18
LOFT                      18
SERVICE_FLAT              16
CASTLE                     9
OTHER_PROPERTY             9
KOT                        6
Name: count, dtype: int64

**Number of rooms**

In [34]:
number_rooms = analyze_column(df_clean, "number_rooms")
df_clean["number_rooms"].value_counts()

Analysis for column 'number_rooms':
Number of NaN values: 0
Number of unique values: 24
Percentage of NaN values: 0.00%
Percentage of unique values: 0.18%
Unique values: [ 2.  3.  4.  5.  6.  8. 28. 27.  7.  9.  1.  0. 10. 15. 17. 13. 11. 14.
 12. 25. 16. 18. 21. 19.]


number_rooms
3.0     4705
4.0     2967
2.0     2870
5.0     1324
1.0      695
6.0      503
7.0      213
0.0      172
8.0      116
9.0       43
10.0      33
12.0      16
11.0      13
13.0       9
15.0       7
14.0       5
17.0       4
28.0       3
16.0       3
18.0       3
27.0       2
19.0       2
25.0       1
21.0       1
Name: count, dtype: int64

**Price**

In [35]:
price = analyze_column(df_clean, "price")
df_clean["price"].value_counts()
df_clean["price"].describe()

Analysis for column 'price':
Number of NaN values: 0
Number of unique values: 1063
Percentage of NaN values: 0.00%
Percentage of unique values: 7.75%
Unique values: [ 249000.  249900.  259000. ... 1420000. 1645000.   42500.]


count    1.371000e+04
mean     4.841840e+05
std      5.633111e+05
min      2.300000e+04
25%      2.390000e+05
50%      3.450000e+05
75%      5.150000e+05
max      9.000000e+06
Name: price, dtype: float64

**Creating a new column called price_per_sqm**

In [36]:
# Create the new "price_per_sqm" column
df_clean['price_per_sqm'] = (df_clean['price'] / df_clean['living_area']).round(2)


**Count the missing values per column**

In [37]:
df_clean.isna().sum()
#df_clean.shape

url                           0
id                            0
region                        2
province                      2
locality                      0
zip_code                      0
Longitude                     0
Latitude                      0
property_type                 0
property_subtype              0
price                         0
number_rooms                  0
living_area                   0
kitchen                       0
furnished                  8433
fireplace                     0
terrace                       0
terrace_area                  0
garden                        0
garden_area                   0
surface_land                  0
number_facades                0
swimming_pool                 0
building_state                0
energy_type                   0
EPC_score                     0
EnergyConsumptionPerSqm       0
parking_outdoor               0
parking_indoor                0
price_per_sqm                 0
dtype: int64

In [38]:
df_clean.to_csv('cleaned_apartment.csv')

**Outliers**

In [39]:
# Filter the DataFrame for rows with "apartment" type in the "property_type" column
df_house = df_clean[df_clean["property_type"] == "HOUSE"]
df_house.shape


(11326, 30)

**Handle outliers per column**

In [40]:
def handle_outliers(df, columns_to_remove_outliers, zscore_threshold=3):
    """
    Handle outliers in specified columns of a DataFrame.
    Parameters:
        df (DataFrame): Input DataFrame.
        columns_to_remove_outliers (list): List of column names to remove outliers.
        zscore_threshold (float): Z-score threshold for identifying outliers. Default is 3.
    Returns:
        DataFrame: DataFrame with outliers removed.
    """
    # Create a copy of the DataFrame to avoid modifying the original DataFrame
    df_house = df.copy()

    # Iterate through each column in columns_to_remove_outliers
    for column_name in columns_to_remove_outliers:
        # Convert the column to a numeric data type, ignoring errors
        df_house[column_name] = pd.to_numeric(df_house[column_name], errors='coerce')

        # Calculate Z-scores for the specified column
        z_scores = (df_house[column_name] - df_house[column_name].mean()) / df_house[column_name].std()


        # Create a mask to identify outliers
        outlier_mask = np.abs(z_scores) > zscore_threshold

        # Print the columns for which outliers are being removed
        print("Removing outliers for column: " + column_name)

        # Count missing values
        missing_values_count = df_house[column_name].isna().sum()
        print("Number of missing values in " + column_name + ": " + str(missing_values_count))

    return df_house

# Call the modified function
columns_to_remove_outliers = ['price', 'number_rooms', 'living_area', 'number_facades', 'EnergyConsumptionPerSqm']
apartment_filtered = handle_outliers(df_house, columns_to_remove_outliers)

df_house.shape


Removing outliers for column: price
Number of missing values in price: 0
Removing outliers for column: number_rooms
Number of missing values in number_rooms: 0
Removing outliers for column: living_area
Number of missing values in living_area: 0
Removing outliers for column: number_facades
Number of missing values in number_facades: 0
Removing outliers for column: EnergyConsumptionPerSqm
Number of missing values in EnergyConsumptionPerSqm: 0


(11326, 30)

In [41]:
df_house.dtypes

url                         object
id                         float64
region                      object
province                    object
locality                    object
zip_code                   float64
Longitude                   object
Latitude                    object
property_type               object
property_subtype            object
price                      float64
number_rooms               float64
living_area                float64
kitchen                     object
furnished                  float64
fireplace                   object
terrace                      int64
terrace_area               float64
garden                       int64
garden_area                float64
surface_land               float64
number_facades             float64
swimming_pool               object
building_state              object
energy_type                 object
EPC_score                   object
EnergyConsumptionPerSqm    float64
parking_outdoor             object
parking_indoor      

In [42]:
df_house.describe()

,id,zip_code,price,number_rooms,living_area,furnished,terrace,terrace_area,garden,garden_area,surface_land,number_facades,EnergyConsumptionPerSqm,price_per_sqm
count,1.132600e+04,11326.000000,1.132600e+04,11326.000000,11326.000000,4375.000000,11326.000000,11326.000000,11326.000000,11326.000000,11326.000000,11326.000000,1.132600e+04,11326.000000
mean,1.104608e+07,5151.948614,5.189956e+05,3.662988,222.548561,0.036800,0.588204,12.706604,0.431397,307.631379,1242.418594,2.843104,2.181929e+03,2270.282439
std,1.867177e+05,2982.633100,6.032774e+05,1.635495,144.743416,0.188292,0.492180,51.050605,0.495293,2295.746691,9716.884665,0.870928,1.901017e+05,1463.035890
min,7.710166e+06,1000.000000,2.300000e+04,0.000000,16.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,-3.600000e+01,80.360000
25%,1.099903e+07,2340.000000,2.500000e+05,3.000000,141.000000,0.000000,0.000000,0.000000,0.000000,0.000000,170.000000,2.000000,2.110000e+02,1504.472500
50%,1.111734e+07,4520.000000,3.690000e+05,3.000000,184.000000,0.000000,1.000000,0.000000,0.000000,0.000000,400.000000,3.000000,3.170000e+02,2027.950000
75%,1.115800e+07,8310.000000,5.499000e+05,4.000000,255.000000,0.000000,1.000000,15.000000,1.000000,130.000000,944.000000,4.000000,4.590000e+02,2657.062500
max,1.117488e+07,9991.000000,9.000000e+06,28.000000,3255.000000,1.000000,1.000000,3466.000000,1.000000,150000.000000,917440.000000,4.000000,2.023112e+07,30629.920000
